In [4]:
import subprocess
import json

from utils.iter import generate_combinations

In [7]:
def gen_sbatch(config_string):
    sbatch_string = f"""#!/bin/bash

    #SBATCH --nodes=1
    #SBATCH --ntasks-per-node=1
    #SBATCH --cpus-per-task=8
    #SBATCH --time=0:40:00
    #SBATCH --mem=32GB
    #SBATCH --gres=gpu:1
    #SBATCH --job-name=diffeo
    #SBATCH --mail-type=FAIL
    #SBATCH --mail-user=cm6627@nyu.edu
    #SBATCH --output=./slurm/slurm_%j.out
    #SBATCH --error=./slurm/slurm_%j.err
    
    module purge
    
    singularity exec --nv \
      --overlay /scratch/cm6627/diffeo_cnn/my_env/overlay-15GB-500K.ext3:ro \
      --overlay /vast/work/public/ml-datasets/imagenet/imagenet-val.sqf:ro \
      --overlay /vast/work/public/ml-datasets/imagenet/imagenet-test.sqf:ro \
      --overlay /vast/work/public/ml-datasets/imagenet/imagenet-train.sqf:ro \
      /scratch/work/public/singularity/cuda12.3.2-cudnn9.0.0-ubuntu-22.04.4.sif \
      /bin/bash -c "source /ext3/env.sh; python fit.py '{config_string}'" """
    return sbatch_string

def submit(config):
    config_string = str(json.dumps(config)).replace('"', '\\"')
    sbatch_string = gen_sbatch(config_string)

    with open("submit.sbatch", "w") as f:
        f.write(sbatch_string)
    
    subprocess.run(["sbatch ./submit.sbatch"], shell=True)

In [12]:
config = {
    "dropout_rate": [0.2, 0.4, 0.6],
    "batch_size": [16, 32, 64],
    "epochs": 1500,
    "optimizer": {
        "name": "AdamW",
        "lr": [1e-3, 1e-4],
        "weight_decay": [0.02, 0.2, 2.0]
    },
    "scheduler": {
        "name": "StepLR",
        "step_size": 500,
        "gamma": 0.1
    },
    "dataset": "resnet18_imagenet1k_train"
}

for config in generate_combinations(config):
    submit(config)

Submitted batch job 57048254
Submitted batch job 57048255
Submitted batch job 57048256
Submitted batch job 57048257
Submitted batch job 57048258
Submitted batch job 57048259
Submitted batch job 57048260
Submitted batch job 57048261
Submitted batch job 57048262
Submitted batch job 57048263
Submitted batch job 57048264
Submitted batch job 57048265
Submitted batch job 57048266
Submitted batch job 57048267
Submitted batch job 57048268
Submitted batch job 57048269
Submitted batch job 57048270
Submitted batch job 57048271
Submitted batch job 57048272
Submitted batch job 57048273
Submitted batch job 57048274
Submitted batch job 57048275
Submitted batch job 57048276
Submitted batch job 57048277
Submitted batch job 57048278
Submitted batch job 57048279
Submitted batch job 57048280
Submitted batch job 57048281
Submitted batch job 57048282
Submitted batch job 57048283
Submitted batch job 57048284
Submitted batch job 57048285
Submitted batch job 57048286
Submitted batch job 57048287
Submitted batc